Factorisation de matrices non négatives:



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from sklearn.decomposition import NMF, PCA
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import cv2

list_train = []
list_test = []
train_ids_tib_pato = [6,30,45,53,66,71,83,84,87,105,126,139,140,150,152,154,156,158,167,172,183,186,189,194,200,201,207,218,224] #[1,2,3,4,6,12,18]
test_ids_tib_pato = [227,229,230,237,241,245] #[19]
label_train=[]
label_test=[]
train_ids_tib_sain = [1,5,8,10,13,15,16,18,19,20,26,28,33,34,37,38,40,43,44,46,47,52,56,57,61,62,69,93]  #[63,64,65,67,68,69,70]  #7,8,9,49,50,55,57,58,59,60,61,62,
test_ids_tib_sain = [97,104,107,130,135,142] #[71,73,74,75]

datadir='/content/drive/MyDrive/cartes_epaisseur_Standford' 

for i in train_ids_tib_pato:
    image_name = '/Tibia_patho/Tibia_map/Tibia_%03d.png'%(i)
    label_train.append("pathologique")
    list_train.append(datadir + image_name)

for i in test_ids_tib_pato:
    image_name = '/Tibia_patho/Tibia_map/Tibia_%03d.png'%(i)
    label_test.append("pathologique")
    list_test.append(datadir + image_name)

for i in train_ids_tib_sain:
    image_name = '/Tibia/Tibia_map/Tibia_%03d.png'%(i)
    label_train.append("sain")
    list_train.append(datadir + image_name)

for i in test_ids_tib_sain:
    image_name = '/Tibia/Tibia_map/Tibia_%03d.png'%(i)
    label_test.append("sain")
    list_test.append(datadir + image_name)



In [ ]:
i=0
train_mat=np.zeros((90000,len(list_train)))
#train_mat_sain=np.zeros((90000,len(train_ids_tib_sain)))
for img in list_train:
    print(img)  
    img_array=cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    train_mat[:,i]=img_array.flatten()
    i+=1

j=0
test_mat=np.zeros((90000,len(list_test)))
for img in list_test:
    img_array=cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    test_mat[:,j]=img_array.flatten()
    j+=1


/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_006.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_030.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_045.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_053.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_066.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_071.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_083.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_084.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_087.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_105.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibia_patho/Tibia_map/Tibia_126.png
/content/drive/MyDrive/cartes_epaisseur_Standford/Tibi

In [ ]:
model = NMF(n_components=14, init='random', random_state=0)
W_train = model.fit_transform(train_mat)
H_train = model.components_
print(np.dot(np.array(W_train),np.array(H_train)))

H_test=np.dot(np.linalg.inv(W_train.T @ W_train) @ W_train.T , test_mat)

#[-694125.404214712, -378617.5187082729, -52962.619759138375, -36142.35450058719, -49389.040804760305, -20768.601889761234]



[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,


In [ ]:
from sklearn import svm

svc=svm.SVC(probability=True)

svc.fit(H_train.T,label_train)

SVC(probability=True)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

y_pred=svc.predict(H_test.T)
print("The predicted Data is :")
print(y_pred)
print("The actual data is:")
print(np.array(label_test))
print(f"The model is {accuracy_score(label_test,y_pred)*100}% accurate")
print("f1:")
f1_score(label_test,y_pred)
print("precision:")
precision_score(label_test,y_pred)
print("recall:")
recall_score(label_test,y_pred)

The predicted Data is :
['sain' 'pathologique' 'pathologique' 'sain' 'sain' 'sain' 'pathologique'
 'sain' 'sain' 'pathologique' 'sain' 'sain']
The actual data is:
['pathologique' 'pathologique' 'pathologique' 'pathologique'
 'pathologique' 'pathologique' 'sain' 'sain' 'sain' 'sain' 'sain' 'sain']
The model is 50.0% accurate
f1:


ValueError: ignored